Exercise 1. Suppose that the performance measure is concerned with just the first T 
   time steps of the environment and ignores everything thereafter. Show that 
   a rational agent’s action may depend not just on the state of the environment 
   but also on the time step it has reached.

 In cases where the performance measure considers a finite number of steps, the rational action depends not only on the current state but also on the time step. This dependency arises because the remaining number of steps alters the set of possible future rewards. The agent must account for the temporal proximity to the end of the decision horizon to act rationally, optimizing its strategy based on both its current state and how much time it has left to achieve its goals.


Exercise 2. Let us examine the rationality of various vacuum-cleaner agent functions.
1. Show that the simple vacuum-cleaner agent function described in Figure 2.3 is indeed rational under the assumptions listed on page 
2. Describe a rational agent function for the case in which each movement costs one point. Does the corresponding agent program require internal state?
The program must know the current state because it needs to know what is clean/if there is more to clean to optimize efficiency. A variable would be needed to measure the threshold of what should be cleaned. 
3. Discuss possible agent designs for the cases in which clean squares can become dirty and the geography of the environment is unknown. Does it make sense for the agent to learn from its experience in these cases? If so, what should it learn? If not, why not?
The agent would need a loop to continue to explore. You want to minimize the number of movements still so learning the number of iterations before dirt comes back would be helpful. Learning the environment allows for better efficiency. 

In [1]:
''' Implement a performance-measuring environment simulator for the vacuum-cleaner world 
depicted in Figure 2.8 and specified on page . Your implementation should be modular so 
that the sensors, actuators, and environment characteristics (size, shape, dirt placement, 
etc.) can be changed easily. (Note: for some choices of programming language and operating 
system there are already implementations in the online code repository.) ''' 

print ("hello")

hello
